In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from PIL import Image
import random
import platform
import psutil
from urllib.error import URLError
from tabulate import tabulate
import time

# === Set random seed for reproducibility ===
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(42)

# === Device ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# === Hardware information ===
cpu_info = platform.processor()
ram_info = psutil.virtual_memory()
total_ram_gb = ram_info.total / (1024 ** 3)
try:
    gpu_name = torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU available"
except:
    gpu_name = "No GPU available"
print(f"CPU: {cpu_info}")
print(f"Total RAM (GB): {round(total_ram_gb, 2)}")
print(f"GPU: {gpu_name}")

# === Data preparation ===
data_dir = "/kaggle/input/classification-of-rice-varieties-in-bangladesh/An Extensive Image Dataset for Classifying Rice Varieties in Bangladesh/Augmented/Augmented"
file_paths = []
labels = []

if not os.path.exists(data_dir):
    raise FileNotFoundError(f"Dataset directory {data_dir} does not exist.")

for class_name in os.listdir(data_dir):
    class_dir = os.path.join(data_dir, class_name)
    if os.path.isdir(class_dir):
        for image_name in os.listdir(class_dir):
            file_paths.append(os.path.join(class_dir, image_name))
            labels.append(class_name)

df = pd.DataFrame({"file_path": file_paths, "label": labels})
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# === Display class distribution ===
class_counts = df['label'].value_counts()
plt.figure(figsize=(10, 6))
ax = class_counts.plot(kind='bar')
plt.xlabel('Classes')
plt.ylabel('Number of Images')
plt.title('Class Distribution')
plt.xticks(rotation=45, ha='right')
for i, count in enumerate(class_counts):
    ax.text(i, count + 5, str(count), ha='center')
plt.ylim(0, max(class_counts) * 1.2)
plt.tight_layout()
plt.savefig('/kaggle/working/class_distribution.png')  # Save plot
plt.close()

# === Display a random sample image ===
random_index = random.randint(0, len(df) - 1)
random_row = df.iloc[random_index]
image = Image.open(random_row['file_path'])
plt.figure(figsize=(6, 6))
plt.imshow(image)
plt.title(f"Label: {random_row['label']}")
plt.axis('off')
plt.savefig('/kaggle/working/sample_image.png')  # Save plot
plt.close()

# === Train/Validation/Test split ===
train_df, temp_df = train_test_split(df, test_size=0.30, stratify=df['label'], random_state=42)
valid_df, test_df = train_test_split(temp_df, test_size=0.50, stratify=temp_df['label'], random_state=42)

print(f"Training Data: {len(train_df)}")
print(f"Validation Data: {len(valid_df)}")
print(f"Test Data: {len(test_df)}")
print(f"Total Data: {len(df)}")

# === Display split distributions ===
def print_split_distribution(df, title):
    counts = df['label'].value_counts()
    table_data = [[class_name, count] for class_name, count in counts.items()]
    print(f"\n{title}")
    print(tabulate(table_data, headers=["Class", "Count"]))

print_split_distribution(train_df, "Train Dataset")
print_split_distribution(valid_df, "Validation Dataset")
print_split_distribution(test_df, "Test Dataset")

# === Custom Dataset ===
class CustomImageDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform
        self.label_map = {label: idx for idx, label in enumerate(sorted(set(dataframe['label'])))}
        self.class_names = sorted(set(dataframe['label']))

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]['file_path']
        label = self.label_map[self.dataframe.iloc[idx]['label']]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

# === Data transforms ===
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

valid_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# === Datasets and DataLoaders ===
try:
    train_dataset = CustomImageDataset(train_df, transform=train_transform)
    valid_dataset = CustomImageDataset(valid_df, transform=valid_transform)
    test_dataset = CustomImageDataset(test_df, transform=valid_transform)
except Exception as e:
    print(f"Error creating datasets: {e}")
    raise

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

# === Modified Model Initialization with Error Handling ===
def initialize_model(model_name, num_classes):
    try:
        print(f"Attempting to download pretrained weights for {model_name}...")
        if model_name == "convnext_tiny":
            model = models.convnext_tiny(weights='IMAGENET1K_V1')
            num_ftrs = model.classifier[2].in_features
            model.classifier[2] = nn.Linear(num_ftrs, num_classes)
        elif model_name == "mobilenet_v3_small":
            model = models.mobilenet_v3_small(weights='IMAGENET1K_V1')
            num_ftrs = model.classifier[-1].in_features
            model.classifier[-1] = nn.Linear(num_ftrs, num_classes)
        else:
            raise ValueError(f"Unsupported model: {model_name}")
        print("Successfully loaded pretrained weights!")
    except URLError as e:
        print(f"Failed to download pretrained weights: {e}")
        print("Falling back to random initialization...")
        if model_name == "convnext_tiny":
            model = models.convnext_tiny(weights=None)
            num_ftrs = model.classifier[2].in_features
            model.classifier[2] = nn.Linear(num_ftrs, num_classes)
        elif model_name == "mobilenet_v3_small":
            model = models.mobilenet_v3_small(weights=None)
            num_ftrs = model.classifier[-1].in_features
            model.classifier[-1] = nn.Linear(num_ftrs, num_classes)
    return model.to(device)

# === Training function with checkpointing ===
def train_model(model, model_name, criterion, optimizer, scheduler, 
                train_loader, valid_loader, num_epochs=20, 
                early_stop_patience=5, save_path_checkpoints="checkpoints"):
    
    os.makedirs(save_path_checkpoints, exist_ok=True)
    history_csv_path = os.path.join(save_path_checkpoints, f"{model_name}_training_history.csv")
    last_checkpoint_path = os.path.join(save_path_checkpoints, f"{model_name}_last.pt")
    
    # Initialize training state
    start_epoch = 0
    train_loss_history = []
    train_acc_history = []
    val_loss_history = []
    val_acc_history = []
    best_val_acc = 0.0
    consecutive_no_improvement = 0
    num_epochs_loss_greater = 0
    
    # Check for existing checkpoint to resume
    if os.path.exists(last_checkpoint_path):
        print(f"Resuming training from checkpoint: {last_checkpoint_path}")
        try:
            checkpoint = torch.load(last_checkpoint_path, map_location=device)
            model.load_state_dict(checkpoint['model_state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            if 'scheduler_state_dict' in checkpoint:
                scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
            
            start_epoch = checkpoint['epoch'] + 1
            train_loss_history = checkpoint['train_loss_history']
            train_acc_history = checkpoint['train_acc_history']
            val_loss_history = checkpoint['val_loss_history']
            val_acc_history = checkpoint['val_acc_history']
            best_val_acc = checkpoint['best_val_acc']
            consecutive_no_improvement = checkpoint['consecutive_no_improvement']
            num_epochs_loss_greater = checkpoint['num_epochs_loss_greater']
            
            print(f"Resuming from epoch {start_epoch} | Previous best val acc: {best_val_acc:.4f}")
        except Exception as e:
            print(f"Error loading checkpoint: {e}. Starting from scratch.")
    
    # Create or append to history CSV
    if not os.path.exists(history_csv_path) or start_epoch == 0:
        with open(history_csv_path, 'w') as f:
            f.write("Epoch,Train Loss,Train Accuracy,Validation Loss,Validation Accuracy\n")
    else:
        print(f"Appending to existing history: {history_csv_path}")

    # Main training loop
    for epoch in range(start_epoch, num_epochs):
        epoch_start_time = time.time()
        
        # Training
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, predicted = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()
            progress_bar.set_postfix(loss=running_loss/total_train, acc=correct_train/total_train)

        epoch_train_loss = running_loss / total_train
        epoch_train_acc = correct_train / total_train
        train_loss_history.append(epoch_train_loss)
        train_acc_history.append(epoch_train_acc)
        print(f'Training Loss: {epoch_train_loss:.4f} Acc: {epoch_train_acc:.4f}')

        # Validation
        model.eval()
        running_loss = 0.0
        correct_val = 0
        total_val = 0

        with torch.no_grad():
            for inputs, labels in valid_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, predicted = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()

        epoch_val_loss = running_loss / total_val
        epoch_val_acc = correct_val / total_val
        val_loss_history.append(epoch_val_loss)
        val_acc_history.append(epoch_val_acc)
        print(f'Validation Loss: {epoch_val_loss:.4f} Acc: {epoch_val_acc:.4f}')

        # Step scheduler
        if isinstance(scheduler, optim.lr_scheduler.ReduceLROnPlateau):
            scheduler.step(epoch_val_acc)
        else:
            scheduler.step()

        # Save checkpoint after every epoch
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'train_loss_history': train_loss_history,
            'train_acc_history': train_acc_history,
            'val_loss_history': val_loss_history,
            'val_acc_history': val_acc_history,
            'best_val_acc': best_val_acc,
            'consecutive_no_improvement': consecutive_no_improvement,
            'num_epochs_loss_greater': num_epochs_loss_greater
        }
        
        try:
            torch.save(checkpoint, last_checkpoint_path)
            # Append to CSV history
            with open(history_csv_path, 'a') as f:
                f.write(f"{epoch+1},{epoch_train_loss:.4f},{epoch_train_acc:.4f},{epoch_val_loss:.4f},{epoch_val_acc:.4f}\n")
            print(f"Checkpoint saved for epoch {epoch+1}")
        except Exception as e:
            print(f"Error saving checkpoint: {e}")

        # Save best model separately
        if epoch_val_acc > best_val_acc:
            best_val_acc = epoch_val_acc
            best_checkpoint_path = os.path.join(save_path_checkpoints, f"{model_name}_best.pt")
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'val_acc': best_val_acc
            }, best_checkpoint_path)
            print(f"New best model saved with validation accuracy: {best_val_acc:.4f}")
            consecutive_no_improvement = 0
        else:
            consecutive_no_improvement += 1

        # Check for validation loss being greater than training loss
        if epoch_val_loss > epoch_train_loss:
            num_epochs_loss_greater += 1
        else:
            num_epochs_loss_greater = 0

        # Early stopping check
        stop_reason = None
        if consecutive_no_improvement >= early_stop_patience:
            stop_reason = f"Validation accuracy not improving for {early_stop_patience} epochs"
        if num_epochs_loss_greater >= early_stop_patience:
            stop_reason = f"Validation loss > training loss for {early_stop_patience} epochs"
        
        if stop_reason:
            print(f"Early stopping triggered after epoch {epoch+1}: {stop_reason}")
            break

        epoch_time = time.time() - epoch_start_time
        print(f"Epoch {epoch+1} completed in {epoch_time:.2f} seconds")

    return train_loss_history, train_acc_history, val_loss_history, val_acc_history

# === Evaluation function ===
def evaluate_model(model, dataloader, class_names, model_name):
    model.eval()
    all_labels = []
    all_preds = []
    running_loss = 0.0
    total = 0
    criterion = nn.CrossEntropyLoss()

    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc=f"Evaluating {model_name}"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            running_loss += loss.item() * inputs.size(0)
            total += labels.size(0)

    avg_loss = running_loss / total
    avg_acc = (np.array(all_labels) == np.array(all_preds)).mean()
    print(f"\n{model_name} Test Loss: {avg_loss:.4f}")
    print(f"{model_name} Test Accuracy: {avg_acc:.4f}")

    print(f"\n{model_name} Classification Report:")
    print(classification_report(all_labels, all_preds, target_names=class_names))

    cm = confusion_matrix(all_labels, all_preds)
    print(f"\n{model_name} Confusion Matrix:")
    print(cm)

    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'{model_name} Confusion Matrix')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig(f'/kaggle/working/{model_name}_confusion_matrix.png')
    plt.close()

    return all_labels, all_preds, cm, avg_loss, avg_acc

# === Main Execution ===
models_to_train = ["convnext_tiny", "mobilenet_v3_small"]
num_classes = len(set(df['label']))
save_path_checkpoints = "/kaggle/working/checkpoints"
os.makedirs(save_path_checkpoints, exist_ok=True)

# Set cache directory for pretrained models
os.environ['TORCH_HOME'] = '/kaggle/working/pretrained_models'

for model_name in models_to_train:
    print(f"\n{'='*40}")
    print(f"=== Training {model_name} ===")
    print(f"{'='*40}\n")
    
    model = initialize_model(model_name, num_classes)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.1, patience=3, verbose=True
    )

    # Train with checkpointing
    train_loss_history, train_acc_history, val_loss_history, val_acc_history = train_model(
        model, model_name, criterion, optimizer, scheduler, 
        train_loader, valid_loader, num_epochs=50,
        save_path_checkpoints=save_path_checkpoints
    )

    # Plot training history
    if train_loss_history:
        epochs = range(1, len(train_loss_history) + 1)
        plt.figure(figsize=(12, 4))
        
        plt.subplot(1, 2, 1)
        plt.plot(epochs, train_loss_history, 'b-', label='Train Loss')
        plt.plot(epochs, val_loss_history, 'r-', label='Validation Loss')
        plt.title(f'{model_name} Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        
        plt.subplot(1, 2, 2)
        plt.plot(epochs, train_acc_history, 'b-', label='Train Accuracy')
        plt.plot(epochs, val_acc_history, 'r-', label='Validation Accuracy')
        plt.title(f'{model_name} Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()
        
        plt.tight_layout()
        plt.savefig(f'/kaggle/working/{model_name}_training_history.png')
        plt.close()

    # Load best model for evaluation
    best_model_path = os.path.join(save_path_checkpoints, f"{model_name}_best.pt")
    if os.path.exists(best_model_path):
        try:
            checkpoint = torch.load(best_model_path, map_location=device)
            model.load_state_dict(checkpoint['model_state_dict'])
            best_epoch = checkpoint['epoch'] + 1
            print(f"Loaded best model from epoch {best_epoch} with validation accuracy: {checkpoint['val_acc']:.4f}")
        except Exception as e:
            print(f"Error loading best model: {e}. Using final model state.")
    else:
        print(f"No best model found for {model_name}. Using final model state.")

    # Evaluate on test set
    class_names = train_dataset.class_names
    if len(class_names) != num_classes:
        print(f"Warning: Dataset has {len(class_names)} classes, but model expects {num_classes} classes.")
    
    print(f"\n{'='*40}")
    print(f"=== Evaluating {model_name} on Test Set ===")
    print(f"{'='*40}\n")
    
    labels, preds, cm, test_loss, test_acc = evaluate_model(
        model, test_loader, class_names, model_name
    )
    
    # Save evaluation results
    with open(f'/kaggle/working/{model_name}_test_results.txt', 'w') as f:
        f.write(f"Test Loss: {test_loss:.4f}\n")
        f.write(f"Test Accuracy: {test_acc:.4f}\n\n")
        f.write("Classification Report:\n")
        f.write(classification_report(labels, preds, target_names=class_names))
        f.write("\nConfusion Matrix:\n")
        f.write(str(cm))

print("\nTraining and evaluation completed successfully!")
print("All checkpoints and results saved to /kaggle/working")

Using device: cuda
CPU: x86_64
Total RAM (GB): 31.35
GPU: Tesla P100-PCIE-16GB
Training Data: 53200
Validation Data: 11400
Test Data: 11400
Total Data: 76000

Train Dataset
Class         Count
----------  -------
BR22           1400
BRRI67         1400
Binadhan14     1400
Binadhan25     1400
BD57           1400
BRRI102        1400
BD30           1400
BD33           1400
BD70           1400
Binadhan7      1400
BR23           1400
Binadhan17     1400
BD95           1400
BD72           1400
Binadhan20     1400
Binadhan8      1400
BD91           1400
BD93           1400
BD79           1400
BD51           1400
BD75           1400
Binadhan11     1400
BD52           1400
BD49           1400
Binadhan26     1400
Binadhan19     1400
BD76           1400
BD87           1400
BD56           1400
Binadhan21     1400
Binadhan12     1400
BD39           1400
Binadhan16     1400
BD85           1400
Binadhan24     1400
Binadhan23     1400
BRRI74         1400
Binadhan10     1400

Validation Dataset
Class  

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Successfully loaded pretrained weights!


Epoch 1/50: 100%|██████████| 1663/1663 [20:05<00:00,  1.38it/s, acc=0.801, loss=0.648]

Training Loss: 0.6476 Acc: 0.8009


...

Validation Loss: 0.0511 Acc: 0.9824
Checkpoint saved for epoch 10
Epoch 10 completed in 1251.84 seconds


Epoch 11/50: 100%|██████████| 1663/1663 [20:06<00:00,  1.38it/s, acc=0.989, loss=0.0341]

Training Loss: 0.0341 Acc: 0.9892


Validation Loss: 0.0394 Acc: 0.9854
Checkpoint saved for epoch 11
Epoch 11 completed in 1253.14 seconds


Epoch 12/50:  44%|████▍     | 735/1663 [08:53<11:13,  1.38it/s, acc=0.989, loss=0.0335]